In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
from torchvision import models

In [4]:
dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'quantization',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'utils',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vg

In [5]:
alexnet = models.AlexNet()

In [6]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [12]:
resnet = models.resnet101(pretrained=False)
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
resnet.load_state_dict(torch.load("resnet.pth"))

<All keys matched successfully>

In [18]:
from torchvision import transforms

# sort of like putting layers of transformations together
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(254),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [19]:
from PIL import Image
img = Image.open("dog.jpeg")
img.show()

In [20]:
img_p = preprocess(img)
img_p.shape

torch.Size([3, 254, 254])

In [21]:
img_r = torch.unsqueeze(img_p, 0)
img_r.shape
# 1 dimension added at beginning to refer to the number of examples in the data

torch.Size([1, 3, 254, 254])

In [22]:
resnet.eval() # important for the model to predict properly
# eval basically tells the model that some layers such as dropout
# and batchnorm are not to be used while evaluating
out = resnet(img_r)
out.shape

torch.Size([1, 1000])

In [23]:
with open('data_labels.txt') as f:
    labels = [line.strip() for line in f.readlines()]
len(labels)

1000

In [24]:
values, idx = torch.max(out, 1)
idx = idx[0]
idx

tensor(208)

In [26]:
confidence = torch.nn.functional.softmax(out, dim=1)[0] * 100
whatami = labels[idx]
me = confidence[idx]
print(f"This is {whatami} with {torch.round(me)}% confidence.")

This is Labrador retriever with 65.0% confidence.


In [ ]:
confidence[idx]

In [31]:
sorted_probs, idx = torch.sort(out, descending=True)
print(sorted_probs.shape)
sorted_probs, idx

torch.Size([1, 1000])


(tensor([[ 1.2072e+01,  1.1139e+01,  8.8551e+00,  8.1274e+00,  7.7615e+00,
           7.3783e+00,  6.9491e+00,  6.4673e+00,  6.3788e+00,  6.2375e+00,
           6.2141e+00,  6.1364e+00,  5.9977e+00,  5.7828e+00,  5.7228e+00,
           5.6942e+00,  5.6870e+00,  5.6327e+00,  5.4940e+00,  5.4255e+00,
           5.4205e+00,  5.4011e+00,  5.3511e+00,  5.2123e+00,  5.1683e+00,
           5.1664e+00,  5.1210e+00,  5.0803e+00,  4.9229e+00,  4.8986e+00,
           4.8668e+00,  4.8529e+00,  4.7490e+00,  4.7030e+00,  4.6923e+00,
           4.6100e+00,  4.5872e+00,  4.5421e+00,  4.5225e+00,  4.4915e+00,
           4.4867e+00,  4.4523e+00,  4.4471e+00,  4.4452e+00,  4.4408e+00,
           4.4152e+00,  4.4091e+00,  4.3988e+00,  4.3151e+00,  4.1824e+00,
           4.1171e+00,  4.0739e+00,  4.0501e+00,  4.0359e+00,  4.0090e+00,
           3.9414e+00,  3.9248e+00,  3.9025e+00,  3.7913e+00,  3.7253e+00,
           3.7179e+00,  3.7064e+00,  3.7046e+00,  3.6949e+00,  3.6849e+00,
           3.6848e+00,  3

In [34]:
top5conf = [p for p in sorted_probs[:5]]
top5labels = [l for l in labels[[i[0] for i in idx[:5]]]]
top5conf, top5labels

TypeError: list indices must be integers or slices, not list

In [93]:
torch.save(resnet.state_dict(), "resnet.pth")